### EJERCICIO 3 - AVANZADO

El fichero hr_train.cvs contiene una tabla sobre los empleados de una empresa. Entre otros incluye informacion de su nivel de satisfaccion, de la ultima evaluacion obtenida, del numero de proyecto en el que se encuentra, del numero medio de horas por semana, del numero de años que lleva en la compañia, si ha tenido o no un accidente en el trabajo, si se ha ido (left) de la compañia, si ha tenido una promocion en los ultimos 5 años, y por ultimo su nivel salarial (descrito como medio, alto o bajo) y el departamento en que se encuentra (sales/technnical ...)

El departamento de RRHH esta solicitando un clasificador que pueda predecir si alguien esta en riesgo de irse de la compañia para poder actuar  proactivamente. 

Notas:
- La columna "left" indica si alguien se ha ido de la compañia, es la "y" del problema y el objetivo de la prediccion.
- Las columnas "sales" y "salary" son alfanumericas. La mayoria de clasificadores (aunque no todos) no trabajn bien con este tipo de variables y requieren valores numericos. Dado el caso dispones de diferentes alternativas: (1) eliminar esas columnas y trabajar con el resto o (2) traducir esas columnas en columnas binarias - por ejemplo si "salary" tiene tres valores eliminar la columna original y sustituirla por tres columnas salary_low, salary_medium, salary_high donde se pondra un 1 en el tipo de salario y un 0 en las otras dos columnas restantes. Lo mismo para "sales", donde se transformara esa columna en tantas columnas como departamentos y un 1 en el departamento en que se encuentre el trabajador. Este es un preproceso muy tipico.


Nota de evaluacion:

En este ejercicio mas que el resultado final (que tambien) se evaluara los pasos y el razonamiento utilizado en cada decision que se toma para la implementacion del modelo. 

In [1]:
import pandas as pd
import numpy as np

file1=r'hr_train.csv'

ci=pd.read_csv(file1)
ci.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.42,0.46,2,150,3,0,1,0,sales,medium
1,0.66,0.77,2,171,2,0,0,0,technical,medium
2,0.55,0.49,5,240,3,0,0,0,technical,high
3,0.22,0.88,4,213,3,1,0,0,technical,medium
4,0.20,0.72,6,224,4,0,1,0,technical,medium


In [11]:
ci.left.value_counts()

0    7424
1    3075
Name: left, dtype: int64

In [2]:
ci.salary.value_counts()

low       5104
medium    4515
high       880
Name: salary, dtype: int64

In [3]:
ci.sales.value_counts()

sales          2935
technical      1890
support        1556
IT              825
product_mng     639
marketing       614
hr              532
RandD           530
accounting      527
management      451
Name: sales, dtype: int64